In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time


def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False):
    if debugflag:
        model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(2,1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        return model
        
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        layer_type = layer_config[0]

        if layer_type == 'dense':
            neurons, activation = layer_config[1], layer_config[2]
            if len(layer_config) > 3 and layer_config[3] == 'input':
                input_shape = layer_config[4]
                model.add(layers.Dense(neurons, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(layers.Dense(neurons, activation=activation))
        elif layer_type == 'dropout':
            rate = layer_config[1]
            model.add(layers.Dropout(rate))
        elif layer_type == 'activation':
            activation = layer_config[1]
            model.add(layers.Activation(activation))
        elif layer_type == 'conv2d':
            filters, kernel_size, strides, activation = layer_config[1], layer_config[2], layer_config[3], layer_config[4]
            model.add(layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, activation=activation))
        elif layer_type == 'maxpool2d':
            pool_size = layer_config[1]
            model.add(layers.MaxPool2D(pool_size=pool_size))
        elif layer_type == 'rnn':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences))
        elif layer_type == 'lstm':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.LSTM(units, return_sequences=return_sequences))

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, y_train, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2", total_epoch=50,ea=False,eamonitor="val_loss"):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor=eamonitor, patience=7, min_delta=0.001, verbose=1)


    start_time = time.time()
    if ea:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback,early_stopping])
    else:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2",threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test,threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    #threshold = 0.5  # Define the threshold
    interpredictions=predictions.flatten()
    correct_predictions = np.sum(np.abs(true_labels - interpredictions) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, interpredictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, interpredictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), interpredictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }


In [2]:
# Load dataset
dataset_name = "normal_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.2901 - mae: 0.5012 - val_loss: 0.2500 - val_mae: 0.4996
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2510 - mae: 0.4996 - val_loss: 0.2502 - val_mae: 0.4998
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2514 - mae: 0.5001 - val_loss: 0.2507 - val_mae: 0.4999
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2510 - mae: 0.4996 - val_loss: 0.2501 - val_mae: 0.4990
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2512 - mae: 0.5002 - val_loss: 0.2507 - val_mae: 0.4997
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 984us/step - loss: 0.2508 - mae: 0.5000 - val_loss: 0.2508 - val_mae: 0.4991
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 987us/step - loss: 0.2510 - mae: 0.4997 - val_loss: 0.2507 - val_mae: 0.5002
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - loss: 0.2506 - mae: 0.4994 - val_loss: 0.2501 - val_mae: 0.4994
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/ste

In [3]:
# Load dataset
dataset_name = "binomial_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2686 - mae: 0.4973 - val_loss: 0.2504 - val_mae: 0.5002
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2538 - mae: 0.4992 - val_loss: 0.2506 - val_mae: 0.5003
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2541 - mae: 0.4995 - val_loss: 0.2544 - val_mae: 0.5011
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2517 - mae: 0.4992 - val_loss: 0.2501 - val_mae: 0.4999
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2522 - mae: 0.4999 - val_loss: 0.2540 - val_mae: 0.5010
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2519 - mae: 0.4996 - val_loss: 0.2508 - val_mae: 0.4994
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2516 - mae: 0.4991 - val_loss: 0.2505 - val_mae: 0.5002
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 997us/step - loss: 0.2513 - mae: 0.4999 - val_loss: 0.2506 - val_mae: 0.4997
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 976us/step - loss: 0.2

In [4]:
# Load dataset
dataset_name = "exponential_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2811 - mae: 0.5017 - val_loss: 0.2592 - val_mae: 0.5003
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - loss: 0.2532 - mae: 0.4995 - val_loss: 0.2558 - val_mae: 0.5003
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2520 - mae: 0.4997 - val_loss: 0.2506 - val_mae: 0.4998
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 975us/step - loss: 0.2512 - mae: 0.4999 - val_loss: 0.2506 - val_mae: 0.5000
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - loss: 0.2512 - mae: 0.4994 - val_loss: 0.2503 - val_mae: 0.5000
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - loss: 0.2520 - mae: 0.4998 - val_loss: 0.2518 - val_mae: 0.4999
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 978us/step - loss: 0.2514 - mae: 0.4998 - val_loss: 0.2517 - val_mae: 0.5001
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step - loss: 0.2512 - mae: 0.5002 - val_loss: 0.2510 - val_mae: 0.5001
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 951us/step -

In [5]:
# Load dataset
dataset_name = "gamma_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2998 - mae: 0.5072 - val_loss: 0.2511 - val_mae: 0.5003
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - loss: 0.2525 - mae: 0.4997 - val_loss: 0.2519 - val_mae: 0.4998
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - loss: 0.2555 - mae: 0.5004 - val_loss: 0.2503 - val_mae: 0.5001
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 988us/step - loss: 0.2524 - mae: 0.4992 - val_loss: 0.2515 - val_mae: 0.4999
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - loss: 0.2527 - mae: 0.4998 - val_loss: 0.2502 - val_mae: 0.4999
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - loss: 0.2521 - mae: 0.4988 - val_loss: 0.2502 - val_mae: 0.4998
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 988us/step - loss: 0.2528 - mae: 0.5002 - val_loss: 0.2504 - val_mae: 0.5000
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step - loss: 0.2533 - mae: 0.5012 - val_loss: 0.2571 - val_mae: 0.4994
Epoch 8: early stopping
125/125 ━━━━━━━━━━━━━━━━━━━━ 

In [6]:
# Load dataset
dataset_name = "poisson_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
model = tf.keras.Sequential([
            layers.SimpleRNN(32, activation='relu', input_shape=(dset_features.shape[1],1)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
            ])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='RNN',dataset_name=dataset_name,total_epoch=50,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='RNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2786 - mae: 0.4982 - val_loss: 0.2521 - val_mae: 0.5005
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - loss: 0.2535 - mae: 0.5007 - val_loss: 0.2502 - val_mae: 0.4998
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 987us/step - loss: 0.2531 - mae: 0.5006 - val_loss: 0.2570 - val_mae: 0.5013
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - loss: 0.2519 - mae: 0.5001 - val_loss: 0.2504 - val_mae: 0.5000
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - loss: 0.2516 - mae: 0.5002 - val_loss: 0.2503 - val_mae: 0.4997
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - loss: 0.2517 - mae: 0.5002 - val_loss: 0.2501 - val_mae: 0.5001
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - loss: 0.2510 - mae: 0.5002 - val_loss: 0.2518 - val_mae: 0.5005
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - loss: 0.2511 - mae: 0.4992 - val_loss: 0.2529 - val_mae: 0.5009
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step